In [6]:
import numpy as np
from numpy.typing import NDArray

In [7]:
def lu_decomposition(A: NDArray[np.float64]) -> tuple[NDArray[np.float64], NDArray[np.float64]]:
    '''
    Perform LU decomposition on an invertible matrix A (A = LU)
    Assumes LU decomposition exists (permutation matrix not required)
    
    params:
        A - (n, n) invertible matrix
    
    returns:
        L - (n, n) unit lower triangular matrix (diagonal entries are 1)
        U - (n, n) upper triangular matrix (row echleon form of A)
    '''
    n = A.shape[0]
    L = np.eye(n, dtype=np.float64)
    U = A.astype(np.float64).copy()
    
    for i in range(n):
        for j in range(i + 1, n):
            if np.isclose(U[i, i], 0):
                raise ValueError('A is singular or nearly singular.')
            
            c = U[j, i] / U[i, i]
            U[j, i:] -= c * U[i, i:]
            L[j, i] = c
    
    return L, U

In [11]:
# test the above fn
# NOT part of question paper
A1 = np.array([
    [1, 2, 2],
    [2, 3, 4],
    [3, 4, 9]
])

L, U = lu_decomposition(A1)
print(L, U, sep='\n\n')

L @ U

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 2. 1.]]

[[ 1.  2.  2.]
 [ 0. -1.  0.]
 [ 0.  0.  3.]]


array([[1., 2., 2.],
       [2., 3., 4.],
       [3., 4., 9.]])

### Solve Ax = b using LU decomposition

Ax = b

A = LU

LUx = b

Put Ux = y

1. Forward substitution - solve Ly = b for y

2. Backward substitution - solve Ux = y for x

In [12]:
def forward_substitution(L: NDArray[np.float64], b: NDArray[np.float64]) -> NDArray[np.float64]:
    '''
    Solves for y (Ly = b)
    
    params:
        L - (n, n) unit lower triangular matrix (diagonal entries are 1)
        b - (n,) constants vector
    
    returns:
        y - (n,) variables vector
    '''
    n = L.shape[0]
    y = np.zeros(n, dtype=np.float64)
    
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    
    return y

In [33]:
# b = np.array([2, 3, 10])
# x = np.linalg.solve(A1, b)

In [34]:
# print(f'x is {x}')
# print(f'b is {b}')

x is [-4.  1.  2.]
b is [ 2  3 10]


In [35]:
# A1 @ x

array([ 2.,  3., 10.])

In [36]:
# y = forward_substitution(L, b)
# y

array([ 2., -1.,  6.])

In [14]:
def backward_substitution(U: NDArray[np.float64], y: NDArray[np.float64]) -> NDArray[np.float64]:
    '''
    Solves for x (Ux = y)
    
    params:
        U - (n, n) upper triangular matrix (row echleon form of A)
        y - (n,)
    
    returns:
        x - (n,) variables vector
    '''
    n = U.shape[0]
    x = np.zeros(n, dtype=np.float64)
    
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1 :], x[i + 1 :])) / U[i, i]
    
    return x

In [38]:
# x = backward_substitution(U, y)
# x

array([-4.,  1.,  2.])

In [47]:
# def lu_solve(A: NDArray[np.float64], b: NDArray[np.float64]) -> NDArray[np.float64]:
#     '''
#     explain it later
#     '''
#     L, U = lu_decomposition(A)
#     y = forward_substitution(L, b)
#     x = backward_substitution(U, y)
    
#     return x

In [48]:
# x = lu_solve(A1, b)
# print(x)

[-4.  1.  2.]


# Solution 1

In [15]:
A = np.array([
    [1, 0, -1, 0],
    [0, 2, -1, -1],
    [-1, 3, 0, 2],
    [0, -1, 2, 1]
], dtype=np.float64)

In [16]:
L, U = lu_decomposition(A)

print(f'L:\n{L}\n\nU:\n{U}')

L:
[[ 1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [-1.   1.5  1.   0. ]
 [ 0.  -0.5  3.   1. ]]

U:
[[  1.    0.   -1.    0. ]
 [  0.    2.   -1.   -1. ]
 [  0.    0.    0.5   3.5]
 [  0.    0.    0.  -10. ]]


In [17]:
L @ U

array([[ 1.,  0., -1.,  0.],
       [ 0.,  2., -1., -1.],
       [-1.,  3.,  0.,  2.],
       [ 0., -1.,  2.,  1.]])

# Solution 2

In [18]:
def lu_sol(L: NDArray[np.float64], U: NDArray[np.float64], b: NDArray[np.float64]) -> NDArray[np.float64]:
    '''
    We have the L and U matrices from the fn lu_decomposition
    Next, we use forward and backward substitutions to solve the system Ax = b for x
    
    params:
        L - (n, n)
        U - (n, n)
        b - (n,)
    
    returns:
        x - (n,)
    '''
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    
    return x

In [19]:
e1 = np.array([1, 0, 0, 0], dtype=np.float64)
x1 = lu_sol(L, U, e1)
print(f'x1:\n{x1}')

x1:
[ 0.9  0.1 -0.1  0.3]


In [20]:
e2 = np.array([0, 1, 0, 0], dtype=np.float64)
x2 = lu_sol(L, U, e2)
print(f'x2:\n{x2}')

x2:
[ 0.5  0.5  0.5 -0.5]


In [21]:
e3 = np.array([0, 0, 1, 0], dtype=np.float64)
x3 = lu_sol(L, U, e3)
print(f'x3:\n{x3}')

x3:
[-0.1  0.1 -0.1  0.3]


In [22]:
e4 = np.array([0, 0, 0, 1], dtype=np.float64)
x4 = lu_sol(L, U, e4)
print(f'x4:\n{x4}')

x4:
[ 0.7  0.3  0.7 -0.1]


# Solution 3

b = (2, 4, 20, 5) transpose

xi is soln to Axi = ei

b = 2e1 + 4e2 + 20e3 + 5e4

Therefore, using superposition prinicple we can write Ax = b as:

A(2x1 + 4x2 + 20x3 + 5x4) = 2e1 + 4e3 + 20e3 + 5e4 = b

In [23]:
x_sol = 2 * x1 + 4 * x2 + 20 * x3 + 5 * x4

print(f'x_sol:\n{x_sol}')

x_sol:
[5.3 5.7 3.3 4.1]


In [24]:
# We can verfiy superposition principle like this:
np.linalg.solve(A, np.array([2, 4, 20, 5], dtype=np.float64))

array([5.3, 5.7, 3.3, 4.1])

# Solution 4

In [25]:
# Frobenius norm
num_fro = np.linalg.norm(A, ord='fro')
print(f'Frobenius norm of A:\n{num_fro}\n')

# p-norm when p = 1
num_p1 = np.linalg.norm(A, ord=1)
print(f'p-norm when p = 1 of A:\n{num_p1}\n')

# p-norm when p = inf
num_pinf = np.linalg.norm(A, ord=np.inf)
print(f'p-norm when p = inf of A:\n{num_pinf}\n')

Frobenius norm of A:
5.291502622129181

p-norm when p = 1 of A:
6.0

p-norm when p = inf of A:
6.0



# Solution 5

In [26]:
# condition number of A wrt 1-norm
k1 = np.linalg.cond(A, p=1)
print(f'Condition number of the matrix A with respect to the 1-norm:\n{k1}\n')

Condition number of the matrix A with respect to the 1-norm:
12.000000000000004



In [27]:
# maxmag(A) = ||A||
mxmag = np.linalg.norm(A, ord=1)
print(f'maxmag(A) with respect to the 1-norm:\n{mxmag}\n`')

maxmag(A) with respect to the 1-norm:
6.0
`


In [28]:
# For minmag(A), we use k(A) = maxmag(A) / minmag(A)
# which implies, minmag(A) = maxmag(A) / k(A)
mnmag = mxmag / k1
print(f'minmag(A) with respect to the 1-norm:\n{mnmag}\n')

# we can also use minmag(A) = 1 / maxmag(A^-1), which gives
mnmag = 1 / np.linalg.norm(np.linalg.inv(A), ord=1)
print(f'minmag(A) with respect to the 1-norm:\n{mnmag}\n')

minmag(A) with respect to the 1-norm:
0.49999999999999983

minmag(A) with respect to the 1-norm:
0.4999999999999999

